In [5]:
pip install faker


   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/1.8 MB ? eta -:--:--
   ----------------- ---------------------- 0.8/1.8 MB 2.4 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.8 MB 2.2 MB/s eta 0:00:01
   ---------------------------------- ----- 1.6/1.8 MB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 1.8 MB/s eta 0:00:00


In [6]:
from faker import Faker
import random

In [7]:
fake  = Faker()

In [34]:
# Data creation for Patient Table

with open('SQL HMS.sql', 'w') as f:
    f.write("INSERT INTO Patient (patient_id, hospital_id, patient_name, dob, admission_datetime, discharge_datetime) VALUES\n")
    
    for patient_id in range(1, 100001):
        hospital_id = random.randint(1, 5)  
        patient_name = fake.name()  
        dob = fake.date_of_birth(minimum_age=0, maximum_age=95)  
        admission_datetime = fake.date_time_between(start_date='-2y', end_date='now')  
        
        
        if random.random() < 0.2:  
            discharge_datetime = 'NULL'
        else:
            discharge_datetime = fake.date_time_between(start_date=admission_datetime, end_date='now')
            discharge_datetime = f"'{discharge_datetime}'"  
        
        f.write(f"({patient_id}, {hospital_id}, '{patient_name}', '{dob}', '{admission_datetime}', {discharge_datetime})")
        
        if patient_id != 100000:
            f.write(",\n")
        else:
            f.write(";\n")

print("SQL INSERT statements for 100,000 records have been written to 'SQL HMS.sql'.")

SQL INSERT statements for 100,000 records have been written to 'SQL HMS.sql'.


In [17]:
# Data creation for Daignosis Table

diagnosis_names = [
    "Seasonal Allergies",
    "Common Cold",               
    "Flu",                        
    "Mild Asthma",                
    "Tension Headache",           
    "Hypertension",               
    "Diabetes Type 2",            
    "Chronic Fatigue Syndrome",   
    "Gastroesophageal Reflux Disease",  
    "Anxiety Disorder",           
    "Chronic Bronchitis",         
    "Hyperlipidemia",             
    "Depression",                 
    "Sleep Apnea",                
    "Pneumonia",
    "Acid Reflux",                
    "Bipolar Disorder",           
    "Osteoarthritis",             
    "Kidney Stones",              
    "Heart Disease",              
    "Coronary Artery Disease",    
    "Myocardial Infarction",      
    "Stroke",                     
    "Chronic Obstructive Pulmonary Disease (COPD)",
    "Cancer",                     
    "Multiple Sclerosis",         
    "Alzheimer Disease",        
    "HIV/AIDS",                   
    "Lupus",                      
    "Celiac Disease",             
    "Rheumatoid Arthritis",       
    "Fibromyalgia",               
    "Sepsis",                     
    "Thyroid Disorders",          
    "Cirrhosis",                  
    "Tuberculosis",               
]

with open('SQL Project HMS.sql','w') as f:
    f.write("INSERT INTO Diagnosis (diagnosis_id, patient_id, diagnosis_name) VALUES\n")

    diagnosis_id = 1
    num_patients = 100000

    for patient_id in range(1, num_patients + 1):
        for _ in range(2):
            diagnosis_name = random.choice(diagnosis_names)
            f.write(f"({diagnosis_id},{patient_id},'{diagnosis_name}')")

            diagnosis_id += 1

            if diagnosis_id <= 200000:  # Ensure we do not exceed 200,000 total diagnoses
                f.write(",\n")

    while diagnosis_id <= 200000 :
        patient_id = random.randint(1, num_patients)
        diagnosis_name = random.choice(diagnosis_names)
        f.write(f"({diagnosis_id},{patient_id},'{diagnosis_name}')")

        diagnosis_id += 1

        if diagnosis_id <= 200000:
             f.write(",\n")

    f.write(";\n")

print("SQL INSERT statements for 200,000 diagnosis records have been written to 'SQL_Project_HMS_Diagnosis.sql'.")


SQL INSERT statements for 200,000 diagnosis records have been written to 'SQL_Project_HMS_Diagnosis.sql'.


In [23]:
pip install pandas faker

Note: you may need to restart the kernel to use updated packages.


In [26]:
## Data creation for Treatment Table

import pandas as pd
from faker import Faker
import random

fake = Faker()

diagnosis_medicines = {
    "Seasonal Allergies": ["Loratadine", "Fluticasone", "Pseudoephedrine", "Ketotifen", "Montelukast"],
    "Common Cold": ["Phenylephrine", "Diphenhydramine", "Dextromethorphan", "Acetaminophen", "Zinc Supplements"],
    "Flu": ["Oseltamivir", "Ibuprofen", "Guaifenesin", "Chlorpheniramine", "Throat Lozenges"],
    "Mild Asthma": ["Fluticasone", "Albuterol", "Montelukast", "Salmeterol", "Theophylline"],
    "Tension Headache": ["Ibuprofen", "Acetaminophen", "Cyclobenzaprine", "Amitriptyline", "Caffeine"],
    "Hypertension": ["Lisinopril", "Metoprolol", "Amlodipine", "Hydrochlorothiazide", "Losartan"],
    "Diabetes Type 2": ["Metformin", "Glipizide", "Sitagliptin", "Liraglutide", "Insulin"],
    "Chronic Fatigue Syndrome": ["Fluoxetine", "Ibuprofen", "Zolpidem", "Coenzyme Q10", "Diphenhydramine"],
    "Gastroesophageal Reflux Disease": ["Omeprazole", "Ranitidine", "Calcium Carbonate", "Metoclopramide", "Alginate-based formulations"],
    "Anxiety Disorder": ["Sertraline", "Venlafaxine", "Lorazepam", "Buspirone", "Propranolol"],
    "Chronic Bronchitis": ["Ipratropium", "Prednisone", "Guaifenesin", "Azithromycin", "Oxygen therapy"],
    "Hyperlipidemia": ["Atorvastatin", "Fenofibrate", "Niacin", "Omega-3 Fatty Acids", "Ezetimibe"],
    "Depression": ["Escitalopram", "Duloxetine", "Bupropion", "Lithium", "Quetiapine"],
    "Sleep Apnea": ["CPAP", "Oral Appliances", "Weight Loss Medications", "Fluticasone", "Positional Therapy Aids"],
    "Pneumonia": ["Amoxicillin", "Oseltamivir", "Dextromethorphan", "Albuterol", "Prednisone"],
    "Acid Reflux": ["Esomeprazole", "Famotidine", "Tums", "Metoclopramide", "Alginate-based formulations"],
    "Bipolar Disorder": ["Lithium", "Lamotrigine", "Aripiprazole", "Fluoxetine", "Clonazepam"],
    "Osteoarthritis": ["Naproxen", "Acetaminophen", "Triamcinolone", "Hyaluronic Acid Injections", "Capsaicin"],
    "Kidney Stones": ["Ibuprofen", "Tamsulosin", "Potassium Citrate", "Ondansetron", "Hydrochlorothiazide"],
    "Heart Disease": ["Rosuvastatin", "Aspirin", "Carvedilol", "Enalapril", "Valsartan"],
    "Coronary Artery Disease": ["Atorvastatin", "Clopidogrel", "Bisoprolol", "Ramipril", "Nitroglycerin"],
    "Myocardial Infarction": ["Aspirin", "Alteplase", "Metoprolol", "Lisinopril", "Atorvastatin"],
    "Stroke": ["Warfarin", "Aspirin", "Reteplase", "Amlodipine", "Atorvastatin"],
    "Chronic Obstructive Pulmonary Disease (COPD)": ["Salbutamol", "Budesonide", "Roflumilast", "Acetylcysteine", "Azithromycin"],
    "Cancer": ["Doxorubicin", "Trastuzumab", "Pembrolizumab", "Tamoxifen", "Morphine"],
    "Multiple Sclerosis": ["Interferon Beta", "Methylprednisolone", "Baclofen", "Gabapentin", "Modafinil"],
    "Alzheimer's Disease": ["Donepezil", "Memantine", "Sertraline", "Rivastigmine", "Galantamine"],
}

treatment_data = []

num_patients = 100000

for patient_id in range(1, num_patients + 1):
    diagnosis = random.choice(list(diagnosis_medicines.keys()))
    medicines = diagnosis_medicines[diagnosis]  
    selected_medicines = random.sample(medicines, 5)  
    
    for medicine_name in selected_medicines:
        dose_time = fake.time()
        duration = random.choice(["1 day", "5 days", "1 week", "2 weeks", "1 month"])
        
        treatment_data.append({
            "treatment_id": len(treatment_data) + 1,  
            "medicine_name": medicine_name,
            "dose_time": str(dose_time),  
            "duration": duration
        })

treatment_df = pd.DataFrame(treatment_data)

# Save DataFrame to CSV or display it
treatment_df.to_csv('treatment_data.csv', index=False)  # Save to CSV file
print(treatment_df.head())  # Show the first few rows

   treatment_id  patient_id     medicine_name dose_time duration
0             1           1       Guaifenesin  11:41:30  1 month
1             2           1  Chlorpheniramine  00:34:01   1 week
2             3           1       Oseltamivir  03:05:12   5 days
3             4           1         Ibuprofen  19:25:28   5 days
4             5           1   Throat Lozenges  02:17:58  2 weeks


In [25]:
treatment_df

,treatment_id,patient_id,medicine_name,dose_time,duration
0,1,1,Dextromethorphan,08:51:58,2 weeks
1,2,1,Diphenhydramine,15:53:19,1 week
2,3,1,Phenylephrine,01:16:41,1 day
3,4,1,Acetaminophen,09:58:07,5 days
4,5,1,Zinc Supplements,08:07:39,1 month
...,...,...,...,...,...
499995,499996,100000,Galantamine,05:59:31,1 day
499996,499997,100000,Sertraline,17:33:35,2 weeks
499997,499998,100000,Memantine,10:20:38,1 day
499998,499999,100000,Donepezil,23:33:06,2 weeks


In [33]:
#Data creation for Billing Table

fake = Faker()

patient_ids = list(range(1, 100001))
random.shuffle(patient_ids)

with open('SQL hms.sql', 'w') as f:
    f.write("INSERT INTO billing (bill_id,patient_id, bill_amount, payment_mode) VALUES\n")
    
    for bill_id in range(1, 100001):
        patient_id = patient_ids[bill_id - 1]  
        bill_amount = round(random.uniform(100, 2000), 2)  
        payment_mode = random.choice(['cash', 'credit'])  
        
        f.write(f"({bill_id},{patient_id}, {bill_amount}, '{payment_mode}')")
        
        # Add a comma after each statement except the last one
        if bill_id != 100000:
            f.write(",\n")
        else:
            f.write(";\n")

print("SQL INSERT statements for 100,000 records have been written to 'SQL HMS.sql'.")

SQL INSERT statements for 100,000 records have been written to 'SQL HMS.sql'.
